# 퍼셉트론
우리가 첫번째로 실습할 딥러닝 예제는 퍼셉트론입니다. 퍼셉트론은 딥러닝은 아니지만, 딥러닝의 뉴론과 상당히 닮은 분류기입니다.  
뉴론과 퍼셉트론은 활성화함수(WX+B)을 출력한다는 점에서 동일합니다. 단 뉴론은 활성화함수를 자유롭게 선택할 수 있는 반면,  
퍼셉트론은 스텝함수를 활성화 함수로 사용합니다.

뉴론: 활성화함수(가중치 * 입력 + 편향값)  
퍼셉트론: 스텝함수(가중치 * 입력 + 편향값)  

퍼셉트론을 이해함으로써, 뉴론의 작동원리를 습득하고, 뉴론 하나만으로도 AND, OR 연산 같은 기초적인 분류를 할 수 있음을 실습을 통해 알아보겠습니다.  
또한 뉴론 하나만으로는 XOR 연산과 같이 선형 분리되지 않는 문제를 풀 수 없음을 직접 실습을 통해 알아보겠습니다.

# 뉴론의 구조
뉴론의 구조는 아래와 같습니다.  
퍼셉트론 역시 같은 구조를 사용합니다. 단 퍼셉트론은 활성화함수로 오로지 스텝함수만을 사용합니다.

![Image of Perceptron](https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/perceptron_structure.png)

# 퍼셉트론 실습
실습에 앞서, 먼저 텐서플로우를 임포트합니다.

In [1]:
import tensorflow as tf

# 상수 설정
코드의 이해도를 높히기 위해서, 1은 True, 0은 False, 그리고 편형값(bias)은 1로 지정합니다.

In [2]:
T = 1.0
F = 0.0
bias = 1.0

# AND, OR 연산의 이해
실습에 앞서서, AND, OR 연산의 입력값과 그에 따른 출력값을 알아봅시다.
![Image of Perceptron](https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/truth_table.png)

# 데이터 획득
아래 AND, OR, XOR 실습 데이터를 함수를 호출하여 받으실 수 있습니다.

In [3]:
def get_AND_data():
    X = [
    [F, F, bias],
    [F, T, bias],
    [T, F, bias],
    [T, T, bias]
    ]
    
    Y = [
        [F],
        [F],
        [F],
        [T]
    ]
    
    return X, Y

def get_OR_data():
    X = [
    [F, F, bias],
    [F, T, bias],
    [T, F, bias],
    [T, T, bias]
    ]
    
    Y = [
        [F],
        [T],
        [T],
        [T]
    ]
    
    return X, Y

def get_XOR_data():
    X = [
    [F, F, bias],
    [F, T, bias],
    [T, F, bias],
    [T, T, bias]
    ]
    
    Y = [
        [F],
        [T],
        [T],
        [F]
    ]
    
    return X, Y

실습할 연산을 선택합니다.

In [4]:
X, Y = get_AND_data()
#X, Y = get_OR_data()
#X, Y = get_XOR_data()

# 퍼셉트론 구성하기
논리 연산용 퍼셉트론을 구현해보도록 하겠습니다.  
논리연산을 위한 입력값 X, Y와 편향값 (b)를 받을 것이므로, weight를 [3,1]로 설정합니다.  
3은 세개의 입력을 의미하고, 1은 한개의 뉴론임을 의미합니다.

In [5]:
W = tf.Variable(tf.random_normal([3, 1]))

# 활성화 함수
퍼셉트론은 아래 스텝함수를 활성화함수로 사용합니다.  
step(x) = { 1 if x > 0; 0 otherwise }

In [6]:
def step(x):
    return tf.to_float(tf.greater(x, 0))

# 손실 함수
평균 제곱 오차(MSE)를 손실함수로 사용하도록 하겠습니다.

In [7]:
f = tf.matmul(X, W)
output = step(f)
error = tf.subtract(Y, output)
mse = tf.reduce_mean(tf.square(error))

# 최적화
퍼셉트론은 경사하강법을 사용한 최적화가 불가능합니다.
매번 학습을 진행할 때마다 가중치를 아래의 룰에 맞게 업데이트합니다.  

if target == 1 and activation == 0:  
  w_new = w_old + input  
  
if target == 0 and activation == 1:  
  w_new = w_old - input  

In [8]:
delta = tf.matmul(X, error, transpose_a=True)
train = tf.assign(W, tf.add(W, delta))

# 학습 및 테스트
아래의 코드를 실행시켜 테스트를 진행합니다.

In [9]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    err = 1
    epoch, max_epochs = 0, 20
    while err > 0.0 and epoch < max_epochs:
        epoch += 1
        err = sess.run(mse)
        sess.run(train)
        print('epoch:', epoch, 'mse:', err)
        
    print("\nTesting Result:")
    print(sess.run([output]))

epoch: 1 mse: 0.25
epoch: 2 mse: 0.25
epoch: 3 mse: 0.5
epoch: 4 mse: 0.25
epoch: 5 mse: 0.25
epoch: 6 mse: 0.25
epoch: 7 mse: 0.5
epoch: 8 mse: 0.25
epoch: 9 mse: 0.0

Testing Result:
[array([[0.],
       [0.],
       [0.],
       [1.]], dtype=float32)]


## 퍼셉트론은 오직 선형 분리되는 데이터 집단만을 분류할 수 있습니다.
퍼셉트론 하나는 한개의 의사결정선과 같습니다. 따라서 퍼셉트론은 오직 선형 분리되는 데이터 집단만을 분류 가능합니다.

![Image of Perceptron](https://qph.fs.quoracdn.net/main-qimg-a6c557af4280d1f85cacc66e048e82f3)

다층퍼셉트론을 사용할 경우, 두개의 뉴론을 히든 레이어에 넣어서 XOR 연산을 구현할 수 있습니다.  
이는 두개의 뉴론이 두개의 의사결정선(z1, z2)을 그리기 때문입니다.

z1, z2에 의한 데이터 분포도는 아래와 같습니다.
z1, z2, value  
0,  0,  0  
0,  1,  1  
1,  0,  1  

아래 z1, z2 챠트에서 볼 수 있듯이, 첫번째 히든레이어을 지난 데이터는 이제 선형 분리될 수 있는 데이터입니다.  
두번째 히든레이어에 퍼셉트론을 하나 놓음으로써 최종적으로 데이터를 선형 분리 할 수 있습니다.

![Image of Perceptron](http://cps0715.weebly.com/uploads/7/4/0/3/74035485/8009014_orig.png)